In [3]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.0


In [5]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 22 kB/s 
     |████████████████████████████████| 503 kB 54.6 MB/s 
     |████████████████████████████████| 3.8 MB 38.5 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=005720b09478d352a127ccee95ebd85ec022f841ce9fb09ee0a0376d669b4dff
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [2]:
# 현재 설치되어 있는 tensorflow의 버전이 2.x 버전이예요!
# 그러면 1.15버전은 못 쓰나요???
# 현재 2.x버전을 삭제하고 1.15버전을 설치하는게 가장 쉽고 편한 방법

import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy import stats
import tensorflow as tf

# Raw Data Loading
df = pd.read_csv('/content/drive/MyDrive/Colab임시폴더/bmi.csv', skiprows=3)
# display(df.head())
# print(df.shape)   # (20000, 3)

In [3]:
train_x_data, test_x_data, train_t_data, test_t_data = \
train_test_split(df[['height', 'weight']],
                 df['label'],
                 test_size=0.3,
                 random_state=1,
                 stratify=df['label'])

scaler = MinMaxScaler()
scaler.fit(train_x_data)

norm_train_x_data = scaler.transform(train_x_data)
norm_test_x_data = scaler.transform(test_x_data)

In [4]:
sess = tf.Session()

onehot_train_t_data = sess.run(tf.one_hot(train_t_data, depth=3))  # depth는 class의 개수
onehot_test_t_data = sess.run(tf.one_hot(test_t_data, depth=3))  # depth는 class의 개수

# tensorflow graph를 그려보아요!
X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([2,3]))
b = tf.Variable(tf.random.normal([3]))

# model, Hypothesis
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# session, 초기화
sess.run(tf.global_variables_initializer())

# 반복 학습
# 그런데 반복학습할때 주의해야 할 점이 있어요!!
# 학습데이터의 사이즈가 매우 크면 메모리에 데이터를 한번에 모두 loading할 수 
# 없어요. memory fault나면서 수행이 중지!
# 어떻게 해결해야 하나요? => batch처리를 해야 해요!
num_of_epoch = 1000 # 학습을 위한 전체 epoch 수
num_of_batch = 100   # 한번에 학습할 데이터 량

for step in range(num_of_epoch):
    total_batch = int(norm_train_x_data.shape[0] / num_of_batch)
    
    for i in range(total_batch):
        batch_x = norm_train_x_data[i*num_of_batch:(i+1)*num_of_batch]
        batch_y = onehot_train_t_data[i*num_of_batch:(i+1)*num_of_batch]
        _, loss_val = sess.run([train, loss], 
                               feed_dict={X:batch_x,
                                          T:batch_y})                           
    if step % 100 == 0:
        print('loss value : {}'.format(loss_val))

loss value : 0.8078733682632446
loss value : 0.16453251242637634
loss value : 0.12264742702245712
loss value : 0.10340364277362823
loss value : 0.09187746047973633
loss value : 0.08403818309307098
loss value : 0.07828585058450699
loss value : 0.07384428381919861
loss value : 0.0702868402004242
loss value : 0.06735865771770477


In [5]:
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

result = sess.run(accuracy, feed_dict={X:norm_test_x_data,
                                       T:onehot_test_t_data})
print(result)  # 0.9855

0.9855
